In [1]:
import polars as pl

%load_ext autoreload
%autoreload 2

In [2]:
df = pl.read_csv("/home/grantham/Desktop/windmark/data/transactions.v1.csv")

df.columns = [col.lower().replace("?", "").replace(" ", "_") for col in df.columns]

In [3]:
df.columns

['user',
 'card',
 'year',
 'month',
 'day',
 'time',
 'amount',
 'use_chip',
 'merchant_name',
 'merchant_city',
 'merchant_state',
 'zip',
 'mcc',
 'errors',
 'is_fraud']

In [4]:

(
    df
    .filter(pl.col('user') < 100)
    .select(pl.all(), timeparts=pl.col("time").str.split(":"))
    .select(
        pl.all(),
        hour=pl.col("timeparts").list.first(),
        minute=pl.col("timeparts").list.last(),
    )
    .select(
        pl.all(),
        timestamp=pl.datetime(pl.col("year"), pl.col("month"), pl.col("day"), pl.col("hour"), pl.col("minute")),
    )
    .select(
        "use_chip",
        "merchant_state",
        "merchant_city",
        "mcc",
        "timestamp",
        amount=pl.col("amount").str.strip_prefix("$").str.to_decimal(),
        merchant_name=pl.col("merchant_name").cast(pl.Utf8),
        target=pl.col("is_fraud"),
        event_id=pl.col("timestamp").cast(pl.Utf8),
        sequence_id=pl.col("user"),
    )
    .sort("timestamp")
    .select(
        pl.all(),
        event_order=pl.col("event_id").cum_count().over("sequence_id"),
        timedelta=pl.col("timestamp").sub(pl.col("timestamp").shift()).dt.seconds(),
    )
    .write_parquet("ledger.parquet")
)

/tmp/ipykernel_21467/2803677089.py:30: DeprecationWarning: `seconds` is deprecated. It has been renamed to `total_seconds`.
  timedelta=pl.col("timestamp").sub(pl.col("timestamp").shift()).dt.seconds(),


In [5]:
dicts = [{"a": 1}, {"b": 2}, {"c": 3}, {}, {}]

In [6]:
dicts

[{'a': 1}, {'b': 2}, {'c': 3}, {}, {}]